<a href="https://colab.research.google.com/github/zhreyaa/software-systems/blob/main/Question_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MT2023179 SHREYA CHAVAN

Question 4 Random Forest

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path_link = "/content/football.csv"
dataframe = pd.read_csv(path_link)
dataframe.head()

,sofifa_id,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,...,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,contribution_type
0,158023,L. Messi,93,93,78000000.0,320000.0,34,170,72,73.0,...,20,35,24,6,11,15,14,8,NaN,1
1,188545,R. Lewandowski,92,92,119500000.0,270000.0,32,185,81,21.0,...,35,42,19,15,6,12,8,10,NaN,1
2,20801,Cristiano Ronaldo,91,91,45000000.0,270000.0,36,187,83,11.0,...,24,32,24,7,11,15,14,11,NaN,1
3,190871,Neymar Jr,91,91,129000000.0,270000.0,29,175,68,73.0,...,35,32,29,9,9,15,15,11,NaN,1
4,192985,K. De Bruyne,91,91,125500000.0,350000.0,30,181,70,10.0,...,68,65,53,15,13,5,10,13,NaN,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

dataframe = dataframe.drop(['short_name','club_name','league_name','preferred_foot','work_rate','body_type','player_tags','player_traits'], axis=1)
categorical_columns = dataframe.select_dtypes(include=['object']).columns
labelencoder = LabelEncoder()

for column in categorical_columns:
    dataframe[column] = labelencoder.fit_transform(dataframe[column])

X = dataframe.drop(['contribution_type'], axis=1)
y = dataframe['contribution_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def trainxgboostmodel(X_train, y_train, X_test, y_test, loss_function):
    params = {'objective': 'multi:softmax','num_class': len(y_train.unique()),'eval_metric': ['mlogloss'],'verbosity': 0}

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_test = xgb.DMatrix(X_test, label=y_test)
    model = xgb.train(params, d_train, num_boost_round=100)
    y_pred = model.predict(d_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, f1

losses = ['gini', 'entropy', 'log loss']

for loss in losses:
    print(f"The Loss function : {loss}")
    accuracy, f1 = trainxgboostmodel(X_train, y_train, X_test, y_test, loss)
    print(f"The Test accuracy : {accuracy:.2f}")
    print(f"The F1 score : {f1:.2f}")
    print()


The Loss function : gini
The Test accuracy : 0.90
The F1 score : 0.90

The Loss function : entropy
The Test accuracy : 0.90
The F1 score : 0.90

The Loss function : log loss
The Test accuracy : 0.90
The F1 score : 0.90



MSE and MAE

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

features = ['potential', 'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg',
            'international_reputation', 'pace', 'shooting', 'passing', 'dribbling',
            'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
            'attacking_heading_accuracy', 'attacking_short_passing',
            'attacking_volleys', 'skill_dribbling', 'skill_curve',
            'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
            'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
            'movement_reactions', 'movement_balance', 'power_shot_power',
            'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
            'mentality_aggression', 'mentality_interceptions',
            'mentality_positioning', 'mentality_vision', 'mentality_penalties',
            'mentality_composure', 'defending_marking_awareness',
            'defending_standing_tackle', 'defending_sliding_tackle',
            'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
            'goalkeeping_positioning', 'goalkeeping_reflexes']

X = dataframe[features]
y = dataframe['overall']

numerical_cols = X.select_dtypes(include=['number']).columns
categorical_cols = X.select_dtypes(exclude=['number']).columns

numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),])

categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),('onehot', OneHotEncoder(handle_unknown='ignore')),])

preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols),('cat', categorical_transformer, categorical_cols),])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', RandomForestRegressor(n_estimators=100, random_state=42))])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"The Mean Squared Error : {mse:.2f}")
print(f"The Mean Absolute Error : {mae:.2f}")


The Mean Squared Error : 0.30
The Mean Absolute Error : 0.34
